In [38]:
import csv
import random

documents = []
with open('../data/training/training-data.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        documents.append(row['chunk'])

documents = random.sample(documents, 1000)

In [39]:
documents

['# VwV Verdachtsunabhängige Kontrollen\r\n\r\nVerwaltungsvorschrift des Sächsischen Staatsministeriums des Innern zur Durchführung ereignis- und verdachtsunabhängiger Kontrollen gemäß § 19 Abs. 1 Satz 1 Nr. 5 des Polizeigesetzes des Freistaates Sachsen (VwV Verdachtsunabhängige Kontrollen)\r\n\r\n',
 '## Sächsisches Psychisch-Kranken-Hilfe-Gesetz §\xa012 Versorgungsverpflichtung der Krankenhäuser\r\n\r\n(1) Die Krankenhäuser sind verpflichtet, die einer psychiatrischen Krankenhausbehandlung bedürfenden Patientinnen und Patienten aus einem nach Absatz\xa03 festgelegten Einzugsgebiet, in dem sie ihren gewöhnlichen Aufenthalt haben oder in dem die stationäre Behandlungsbedürftigkeit eingetreten ist, aufzunehmen und zu behandeln. Den gewöhnlichen Aufenthalt hat eine Person dort, wo sie sich unter den Umständen aufhält, die erkennen lassen, dass sie an diesem Ort oder in diesem Gebiet nicht nur vorübergehend verweilt. Der gewöhnliche Aufenthalt ist auch der Aufenthalt in einer anderen stat

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
model_name = "../training/models/german-nq-paraphrase-multilingual-mpnet-base-v2/checkpoint-4560"
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)
collection = client.get_or_create_collection("documents",embedding_function=sentence_transformer_ef)

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=documents, # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[{"source": "array"}]*len(documents), # filter on these!
    ids=[str(i) for i in range(len(documents))], # unique for each doc
)


In [11]:
# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["Wann dürfen Abgeordnete Urlaub nehmen?"], # query text
    n_results=5,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)

results

{'ids': [['4', '22', '91', '71', '54']],
 'embeddings': None,
 'documents': [['## Abgeordnetengesetz §\xa03 \xa0Wahlvorbereitungsurlaub\n\n(1)\xa0Einer Bewerberin oder einem Bewerber um einen Sitz im Landtag oder in der gesetzgebenden Körperschaft eines anderen Landes ist zur Vorbereitung ihrer oder seiner Wahl innerhalb der letzten zwei\xa0Monate vor dem Wahltag auf Antrag Urlaub bis zu zwei\xa0Monaten zu gewähren. Ein Anspruch auf Fortzahlung der Bezüge besteht für die Dauer der Beurlaubung nicht.\n\n(2)\xa0Einer Beamtin oder einem Beamten beziehungsweise einer Richterin oder einem Richter, die oder der sich um einen Sitz im Landtag oder in der gesetzgebenden Körperschaft eines anderen Landes, im Deutschen Bundestag oder im\xa0Europäischen Parlament bewirbt, ist zur Vorbereitung seiner Wahl innerhalb der letzten zwei\xa0Monate vor dem Wahltag auf Antrag Urlaub bis zu zwei\xa0Monaten unter Wegfall der Dienstbezüge zu gewähren. Der Anspruch auf Beihilfe in Krankheits-, Geburts- und Tod

In [36]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

## LLM Configuration
base_url = 'https://mms-hackathon-openai.openai.azure.com/'
api_key = os.getenv("API_KEY")
use_azure = True
model = 'gpt-4o-mini'

def init_client():
    if (use_azure):
        client = openai.AzureOpenAI(
                azure_endpoint=base_url,
                api_key=api_key,
                api_version="2024-08-01-preview"
        )
    else:
        client = openai.OpenAI(
                api_key=api_key,
                base_url=base_url
        )
    return client

def get_answer(prompt, gesetze, question):
    response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "system", "content": gesetze},
                    {"role": "user", "content": question}],
                temperature=0.7,  # Controls randomness
                max_tokens=1000    # Limits response length
            )
    return response.choices[0].message.content


client = init_client()
with open('answer-prompt.txt', 'r', encoding='utf-8') as f:
    prompt = f.read()

def find_gesetze(question):
        gesetze = collection.query(
            query_texts=[question], # query text
            n_results=5)['documents'][0]
        return "\\n".join(gesetze)

def answer(question):
    return get_answer(prompt, find_gesetze(question), question)

In [ ]:
question = 'Wann ist der Bau meines Carports Verfahrensfrei?'
my_answer = answer(question)


Die Ausbildung zum Amtsanwalt dauert mindestens 18 Monate. Diese Ausbildung gliedert sich in verschiedene Abschnitte, darunter ein fachwissenschaftliches Studium von mindestens sechs Monaten und eine fachpraktische Ausbildung von mindestens neun Monaten. Die Ausbildung schließt mit der Amtsanwaltsprüfung ab.

Diese Informationen findest du in der Sächsischen Ausbildungs- und Prüfungsordnung für Amtsanwälte, § 5.

Hier ist der Link zum Gesetzestext: [Sächsische Ausbildungs- und Prüfungsordnung für Amtsanwälte § 5](#).
